# Bag of Words Meets Bags of Popcorn: Use Google's Word2Vec for movie reviews

## Reading in the Data

In [1]:
import pandas as pd
import nltk

In [2]:
# Reading in the data
train = pd.read_csv("../data/bags_of_popcorn/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, \
                    delimiter="\t", quoting=3) # header set as 0 since first line contains column names; quoting=3 tells Python to ignore doubled quotes to prevent errors 

In [3]:
# Making sure that we read the right amount of rows and columns
train.shape

(25000, 3)

In [4]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [5]:
# Exploring the movie reviews
print(train['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

## Data Cleaning and Text Preprocessing
### Removing HTML Markup: The BeautifulSoup Package

In [6]:
# Importing BeautifulSoup into my workspace to remove HTML markups 
from bs4 import BeautifulSoup             

# Initializing the BeautifulSoup object on a single movie review     
example1 = BeautifulSoup(train["review"][0])  

# Printing the raw review and then the output of get_text() for comparison
print(train["review"][0])
print(example1.get_text()) # get_text gives us the text of the review without any tags or markups 

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

### Dealing with Punctuation, Numbers and Stopwords: NLTK and regular expressions

In [7]:
import re

# Use regular expressions to do a find-and-replace
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search
print(letters_only)

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [8]:
# Converting movie reviews to lower case and splitting them into individual words (i.e., tokenisation)
lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()               # Split into words

In [9]:
# Using nltk to get a list of stop words 
from nltk.corpus import stopwords # importing the stop word list
print(stopwords.words("english")) 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
# Removing stop words from "words"
words = [w for w in words if not w in stopwords.words("english")]
print(words)

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

In [11]:
# Normalising words to shorten the lookup
from nltk.stem import PorterStemmer # importing the PorterStemmer from the NLTK kit

ps = PorterStemmer() # assigning the PorterStemmer to a variable named ps 

stemmed_list = []

for word in words:
    stemmed_word = ps.stem(word)
    stemmed_list.append(stemmed_word)

In [12]:
print(stemmed_list)

['stuff', 'go', 'moment', 'mj', 'start', 'listen', 'music', 'watch', 'odd', 'documentari', 'watch', 'wiz', 'watch', 'moonwalk', 'mayb', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'realli', 'cool', 'eighti', 'mayb', 'make', 'mind', 'whether', 'guilti', 'innoc', 'moonwalk', 'part', 'biographi', 'part', 'featur', 'film', 'rememb', 'go', 'see', 'cinema', 'origin', 'releas', 'subtl', 'messag', 'mj', 'feel', 'toward', 'press', 'also', 'obviou', 'messag', 'drug', 'bad', 'kay', 'visual', 'impress', 'cours', 'michael', 'jackson', 'unless', 'remot', 'like', 'mj', 'anyway', 'go', 'hate', 'find', 'bore', 'may', 'call', 'mj', 'egotist', 'consent', 'make', 'movi', 'mj', 'fan', 'would', 'say', 'made', 'fan', 'true', 'realli', 'nice', 'actual', 'featur', 'film', 'bit', 'final', 'start', 'minut', 'exclud', 'smooth', 'crimin', 'sequenc', 'joe', 'pesci', 'convinc', 'psychopath', 'power', 'drug', 'lord', 'want', 'mj', 'dead', 'bad', 'beyond', 'mj', 'overheard', 'plan', 'nah', 'joe', 'pesci', '

In [13]:
# Removing duplicates from "words" and sorting words in alphabetical order 
new_list = list(sorted(set(stemmed_list)))
print(new_list)

['actual', 'alon', 'also', 'anoth', 'anyway', 'attent', 'away', 'bad', 'behind', 'bestest', 'beyond', 'biographi', 'bit', 'bore', 'bottom', 'buddi', 'bunch', 'call', 'came', 'car', 'certain', 'charact', 'cinema', 'close', 'complex', 'consent', 'convinc', 'cool', 'cours', 'crimin', 'danc', 'dead', 'demon', 'differ', 'director', 'documentari', 'door', 'drug', 'dunno', 'egotist', 'eighti', 'either', 'etc', 'ever', 'exclud', 'extrem', 'fact', 'fan', 'featur', 'feel', 'film', 'final', 'find', 'gave', 'get', 'girl', 'give', 'go', 'grace', 'guilti', 'guy', 'hate', 'hmmm', 'hope', 'impress', 'innoc', 'insight', 'iron', 'jackson', 'joe', 'kay', 'kid', 'kiddi', 'know', 'latter', 'let', 'level', 'liar', 'like', 'line', 'listen', 'lord', 'lot', 'made', 'make', 'may', 'mayb', 'messag', 'michael', 'mind', 'minut', 'mj', 'moment', 'moonwalk', 'movi', 'music', 'must', 'nah', 'nice', 'obviou', 'odd', 'one', 'origin', 'overheard', 'part', 'patienc', 'peopl', 'perform', 'pesci', 'plan', 'planet', 'power'

### Putting it all together

In [14]:
# Creating a function that converts a raw review into a string of words 
def review_to_words(raw_review):
    
    # Removing HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    
    # Removing non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    
    # Converting to lower case and splitting into individual words
    words = letters_only.lower().split()                             
    
    # Converting the stop words to a set
    stops = set(stopwords.words("english")) # searching a set is much faster than searching a list since it removes duplicates
    
    # Removing stop words
    meaningful_words = [w for w in words if not w in stops]  
    
    # Normalising words to shorten the lookup
    final_list_of_words = []

    for word in meaningful_words:
        stemmed_word = ps.stem(word)
        final_list_of_words.append(stemmed_word)
        
    # Joining the words back into one string separated by space
    return( " ".join(final_list_of_words)) # making the output easier to use in our Bag of Words 

In [15]:
# Calling the function for a single review
clean_review = review_to_words(train["review"][0])
print(clean_review)

stuff go moment mj start listen music watch odd documentari watch wiz watch moonwalk mayb want get certain insight guy thought realli cool eighti mayb make mind whether guilti innoc moonwalk part biographi part featur film rememb go see cinema origin releas subtl messag mj feel toward press also obviou messag drug bad kay visual impress cours michael jackson unless remot like mj anyway go hate find bore may call mj egotist consent make movi mj fan would say made fan true realli nice actual featur film bit final start minut exclud smooth crimin sequenc joe pesci convinc psychopath power drug lord want mj dead bad beyond mj overheard plan nah joe pesci charact rant want peopl know suppli drug etc dunno mayb hate mj music lot cool thing like mj turn car robot whole speed demon sequenc also director must patienc saint came film kiddi bad sequenc usual director hate work one kid let alon whole bunch perform complex danc scene bottom line movi peopl like mj one level anoth think peopl stay a

In [16]:
# Getting the number of reviews based on the dataframe column size
num_reviews = train["review"].size

# Initializing an empty list to hold the clean reviews
print("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []

# Looping over each review
for i in range(0, num_reviews): # creating an index i that goes from 0 to the length of the movie review list
    if( (i+1)%1000 == 0): # if the index is evenly divisible by 1000, print a message
        print("Review %d of %d\n" % ( i+1, num_reviews )) 
    # Calling our function for each one, and appending the result to the list of
    clean_train_reviews.append(review_to_words(train["review"][i]))

Cleaning and parsing the training set movie reviews...



/opt/homebrew/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



## Creating Features from a Bag of Words

In [17]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initializing the "CountVectorizer" object (scikit-learn's bag of words tool)
vectorizer = CountVectorizer(analyzer = "word", \
                            tokenizer = None,  \
                            preprocessor = None, \
                            stop_words = None, \
                            max_features = 5000)

# Fitting the model and learning the vocabulary; transforming the training data into feature vectors
train_data_features = vectorizer.fit_transform(clean_train_reviews) # the input is a list of strings 
                             
# Converting the result into an np.array since it is easier to work with
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [18]:
# Seeing how the training data array looks like
print(train_data_features.shape)

(25000, 5000)


In [21]:
# Taking a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
#print(vocab)

In [23]:
print(train_data_features)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [33]:
import numpy as np

# Summing up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each word, print the vocabulary word and the number of times it appears in the training set
# for tag, count in zip(vocab, dist):
#     print(count, tag)

## Random Forest

In [37]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initializing a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fitting the forest to the training set, using the bag of words as features and the sentiment labels as the response variable
forest = forest.fit(train_data_features, train["sentiment"])

Training the random forest...


In [36]:
train["sentiment"]

0        1
1        1
2        0
3        0
4        1
        ..
24995    0
24996    0
24997    0
24998    0
24999    1
Name: sentiment, Length: 25000, dtype: int64

## Creating a Submission

In [46]:
# Running the Random Forest model on the test set and create a submission file
# Predicting the sentiment label 
test_data = pd.read_csv('../data/bags_of_popcorn/word2vec-nlp-tutorial/testData.tsv.zip', header=0, \
                    delimiter="\t", quoting=3) # header set as 0 since first line contains column names; quoting=3 tells Python to ignore doubled quotes to prevent errors 

In [47]:
test_data.shape

(25000, 2)

In [48]:
test_data

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."
...,...,...
24995,"""2155_10""","""Sony Pictures Classics, I'm looking at you! S..."
24996,"""59_10""","""I always felt that Ms. Merkerson had never go..."
24997,"""2531_1""","""I was so disappointed in this movie. I am ver..."
24998,"""7772_8""","""From the opening sequence, filled with black ..."


In [ ]:
y = train['

KeyError: 'sentiment'

In [39]:
# # Reading in the test data
# test = pd.read_csv("../data/bags_of_popcorn/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", \
#                    quoting=3 )

# # Verifying that there are 25,000 rows and 2 columns
# print(test.shape)

# # Creating an empty list and appending the clean reviews one by one
# num_reviews = len(test["review"])
# clean_test_reviews = [] 

# print("Cleaning and parsing the test set movie reviews...\n")
# for i in range(0,num_reviews):
#     if((i+1) % 1000 == 0):
#         print("Review %d of %d\n" % (i+1, num_reviews))
#     clean_review = review_to_words(test["review"][i])
#     clean_test_reviews.append(clean_review)

# # Getting a bag of words for the test set, and converting it to a numpy array
# test_data_features = vectorizer.transform(clean_test_reviews)
# test_data_features = test_data_features.toarray()

# # Using the random forest to make sentiment label predictions
# result = forest.predict(test_data_features)

# # Copying the results to a pandas dataframe with an "id" column and a "sentiment" column
# output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# # Use pandas to write the comma-separated output file
# output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

(25000, 2)
               id                                             review
0      "12311_10"  "Naturally in a film who's main themes are of ...
1        "8348_2"  "This movie is a disaster within a disaster fi...
2        "5828_4"  "All in all, this is a movie for kids. We saw ...
3        "7186_2"  "Afraid of the Dark left me with the impressio...
4       "12128_7"  "A very accurate depiction of small time mob l...
...           ...                                                ...
24995   "2155_10"  "Sony Pictures Classics, I'm looking at you! S...
24996     "59_10"  "I always felt that Ms. Merkerson had never go...
24997    "2531_1"  "I was so disappointed in this movie. I am ver...
24998    "7772_8"  "From the opening sequence, filled with black ...
24999  "11465_10"  "This is a great horror film for people who do...

[25000 rows x 2 columns]


In [ ]:
print(test)